# LangGraph Neo4j MCP Agent (Simple)

A simplified notebook for querying a Neo4j database using LangGraph and the Model Context Protocol (MCP).

This version uses a static access token - no automatic token refresh.

## 1. Install Dependencies

**Important:** These are the correct versions as of January 2026. The LangChain ecosystem moved to 1.x in October 2025.

In [ ]:
# Install compatible package versions (LangChain 1.x ecosystem)
# langchain-mcp-adapters requires langchain-core>=1.0.0

%pip install --upgrade --quiet \
    "langchain>=1.2.0,<2.0.0" \
    "langgraph>=1.0.0,<2.0.0" \
    "langchain-aws>=1.0.0,<2.0.0" \
    "langchain-mcp-adapters>=0.2.1" \
    "mcp>=1.9.2" \
    "httpx>=0.28.0" \
    "boto3>=1.36.0" \
    "nest-asyncio>=1.6.0"

print("Done! Now run the next cell to restart the kernel.")

## 2. Restart Kernel

Run this cell after installing packages. Wait for kernel to restart, then **skip to Section 3**.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 3. Imports

Run this cell after the kernel restarts.

In [ ]:
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# LangChain 1.x imports
from langchain_aws import ChatBedrockConverse
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.client import MultiServerMCPClient

print("Imports complete.")

### Verify Package Versions

In [ ]:
import importlib.metadata

packages = [
    ("langchain", ">=1.2.0"),
    ("langgraph", ">=1.0.0"),
    ("langchain-aws", ">=1.0.0"),
    ("langchain-mcp-adapters", ">=0.2.1"),
    ("mcp", ">=1.9.2"),
]

print("Package Versions:")
print("-" * 50)
for pkg, required in packages:
    try:
        version = importlib.metadata.version(pkg)
        print(f"{pkg:30} {version:15} (need {required})")
    except importlib.metadata.PackageNotFoundError:
        print(f"{pkg:30} NOT INSTALLED")

## 4. Configuration

### **ACTION REQUIRED**

Open your `.mcp-credentials.json` file and copy these values:

```json
{
  "gateway_url": "<-- copy to GATEWAY_URL",
  "access_token": "<-- copy to ACCESS_TOKEN"
}
```

> **Note**: Tokens typically expire after 1 hour. Get a fresh token if you see auth errors.

In [ ]:
# =============================================================================
# REPLACE THESE VALUES with your credentials from .mcp-credentials.json
# =============================================================================

GATEWAY_URL = "YOUR_GATEWAY_URL_HERE"
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN_HERE"

# =============================================================================
# AWS Bedrock settings
# =============================================================================

AWS_REGION = "us-west-2"
MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"

# Validate
if "YOUR_" in GATEWAY_URL or "YOUR_" in ACCESS_TOKEN:
    print("ERROR: Replace GATEWAY_URL and ACCESS_TOKEN above!")
    print("       Get values from .mcp-credentials.json")
else:
    print(f"Gateway: {GATEWAY_URL[:60]}...")
    print(f"Token:   {ACCESS_TOKEN[:30]}...")
    print(f"Region:  {AWS_REGION}")
    print(f"Model:   {MODEL_ID}")
    print("\nConfiguration OK!")

## 5. System Prompt

In [ ]:
SYSTEM_PROMPT = """You are a helpful Neo4j database assistant with access to tools that let you query a Neo4j graph database.

Your capabilities include:
- Retrieve the database schema to understand node labels, relationship types, and properties
- Execute read-only Cypher queries to answer questions about the data
- Do not execute any write Cypher queries

When answering questions about the database:
1. First retrieve the schema to understand the database structure
2. Formulate appropriate Cypher queries based on the actual schema
3. If a query returns no results, explain what you looked for and suggest alternatives
4. Format results in a clear, human-readable way
5. Cite the actual data returned in your response

Important Cypher notes:
- Use MATCH patterns that align with the actual schema
- For counting, use MATCH (n:Label) RETURN count(n)
- For listing items, add LIMIT to avoid overwhelming results
- Handle potential NULL values gracefully

Be concise but thorough in your responses."""

## 6. Create Agent

This uses `ChatBedrockConverse` (recommended for Bedrock) and `create_react_agent` from LangGraph.

In [ ]:
# Global variables to hold client and agent
mcp_client = None
agent = None

async def setup_agent():
    """Initialize the MCP client and create the ReAct agent."""
    global mcp_client, agent
    
    print("Connecting to MCP server...")
    
    # Create MCP client
    mcp_client = MultiServerMCPClient(
        {
            "neo4j": {
                "transport": "streamable_http",
                "url": GATEWAY_URL,
                "headers": {
                    "Authorization": f"Bearer {ACCESS_TOKEN}",
                },
            }
        }
    )
    
    # Get tools from MCP server
    tools = await mcp_client.get_tools()
    print(f"Loaded {len(tools)} tools:")
    for tool in tools:
        print(f"  - {tool.name}")
    
    # Initialize Bedrock LLM (using ChatBedrockConverse directly)
    print(f"\nInitializing LLM: {MODEL_ID}")
    llm = ChatBedrockConverse(
        model=MODEL_ID,
        region_name=AWS_REGION,
        temperature=0,
    )
    
    # Create ReAct agent using LangGraph
    print("Creating ReAct agent...")
    agent = create_react_agent(
        model=llm,
        tools=tools,
        prompt=SYSTEM_PROMPT,
    )
    
    print("Agent ready!")
    return agent

# Run setup
asyncio.get_event_loop().run_until_complete(setup_agent())

## 7. Query Helper

In [ ]:
async def ask(question: str) -> str:
    """Ask the agent a question about the Neo4j database."""
    print("=" * 70)
    print(f"Q: {question}")
    print("=" * 70)
    
    result = await agent.ainvoke(
        {"messages": [("human", question)]}
    )
    
    messages = result.get("messages", [])
    if messages:
        content = getattr(messages[-1], "content", str(messages[-1]))
        print(f"\nA:\n{content}")
        return content
    return "No response"

def query(question: str) -> str:
    """Synchronous wrapper for ask()."""
    return asyncio.get_event_loop().run_until_complete(ask(question))

## 8. Demo Queries

In [ ]:
# Get database schema
_ = query("What is the database schema? Give me a brief summary.")

In [ ]:
# Count nodes
_ = query("How many nodes are in the database by label?")

In [ ]:
# Explore relationships
_ = query("What types of relationships exist in the database?")

## 9. Your Queries

Replace the question and run the cell.

In [ ]:
_ = query("List 5 sample records from the most populated node type.")

In [ ]:
# Add more queries here
# _ = query("Your question here")

---

## Resources

- [LangChain 1.0 Release Notes](https://www.blog.langchain.com/langchain-langgraph-1dot0/)
- [LangChain MCP Adapters](https://github.com/langchain-ai/langchain-mcp-adapters)
- [LangGraph Docs](https://langchain-ai.github.io/langgraph/)
- [ChatBedrockConverse](https://docs.langchain.com/oss/python/integrations/chat/bedrock)